In [1]:
import v20
import pandas as pd
import defs

In [2]:
# create context
ctx = v20.Context(hostname='api-fxpractice.oanda.com', token=defs.key)
ctx.set_header(key='Authorization', value=defs.key)

# streaming context
ctxs = v20.Context(hostname='stream-fxpractice.oanda.com', token=defs.key)
ctxs.set_header(key='Authorization', value=defs.key)

In [3]:
insts = ctx.account.instruments(defs.ACCOUNT_ID).get('instruments')
instruments = {i.name:i.dict() for i in insts}

In [4]:
instruments['EUR_USD']['bid'] = 1.1334

In [5]:
instruments['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1.0',
 'maximumTrailingStopDistance': '1.0',
 'minimumTrailingStopDistance': '0.0005',
 'maximumPositionSize': '0.0',
 'maximumOrderUnits': '100000000.0',
 'marginRate': '0.02',
 'bid': 1.1334}

In [7]:
inst_str=','.join(instruments.keys())
inst_str

'EUR_HUF,EUR_DKK,USD_MXN,GBP_USD,CAD_CHF,EUR_GBP,GBP_CHF,USD_THB,USD_ZAR,EUR_NOK,CAD_JPY,EUR_HKD,AUD_HKD,USD_SEK,GBP_SGD,GBP_HKD,EUR_NZD,SGD_CHF,AUD_SGD,EUR_JPY,USD_CHF,USD_TRY,GBP_JPY,EUR_CZK,CHF_ZAR,EUR_TRY,USD_JPY,GBP_ZAR,SGD_JPY,USD_CZK,USD_NOK,ZAR_JPY,TRY_JPY,USD_DKK,EUR_PLN,AUD_CAD,USD_HKD,CAD_SGD,GBP_NZD,NZD_USD,AUD_NZD,CHF_HKD,NZD_HKD,USD_CNH,NZD_SGD,USD_SGD,EUR_SEK,NZD_CHF,HKD_JPY,AUD_CHF,CHF_JPY,AUD_JPY,EUR_ZAR,EUR_AUD,NZD_JPY,USD_CAD,EUR_CHF,EUR_CAD,USD_HUF,NZD_CAD,EUR_SGD,AUD_USD,EUR_USD,GBP_AUD,USD_PLN,CAD_HKD,GBP_CAD,GBP_PLN'

In [8]:
%%time
prices = ctx.pricing.get(defs.ACCOUNT_ID, instruments=inst_str).get('prices')
price_list = []
for p in prices:
    price_list.append(dict(
        instrument=p.instrument,
        bid=p.bids[0].price,
        ask=p.asks[0].price,
        spread=p.asks[0].price-p.bids[0].price))
df = pd.DataFrame(price_list)    
print(df[df.spread < 0.0003].sort_values(by='spread').to_string(index=False))

Empty DataFrame
Columns: [instrument, bid, ask, spread]
Index: []
CPU times: user 17.6 ms, sys: 3.57 ms, total: 21.2 ms
Wall time: 215 ms


In [9]:
df.head()

,instrument,bid,ask,spread
0,EUR_HUF,368.99500,370.11900,1.12400
1,EUR_DKK,7.43529,7.43736,0.00207
2,USD_MXN,20.60170,20.63116,0.02946
3,GBP_USD,1.33853,1.34003,0.00150
4,CAD_CHF,0.71698,0.71872,0.00174
